# 2.2 ICESat-2 data frame mapping
### taken out of 2.3

In [ ]:
# Exploring filtered ATL08 with plots
print("# of ATL08 columns: ", len(atl08.columns))
print(f"There are {atl08.shape[0]} observations in this dataframe.")
#print("Col names: ", atl08.columns)
print("Max lat:",    atl08.lat.max())
print("Min lat:",    atl08.lat.min())
print("Max lon:",    atl08.lon.max())
print("Min lon:",    atl08.lon.min())
print("Years: ",     atl08.yr.unique())
print("Months: ",    atl08.m.unique())

# Plot obs from night and day
# My cmap
forest_ht_cmap = LinearSegmentedColormap.from_list('forest_ht', ['#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850','#005a32'], 12)

print("\nExample 1: Get only clear sky data, by year, with ht threshold.")

fig, axes = plt.subplots(nrows=1, ncols=atl08.night_flg.nunique(), figsize=(20, 4), sharex=False, sharey=True)
#ax0, ax1, ax2, ax3 = axes.flatten()

SAMP_FRAC = 1 # Map just a portion of the data for speed

i=0
for group_name, group in atl08.groupby('night_flg'):
    print(f"\nThere are {len(group)} observations in {group_name}.")
    #print(f"Filtering for clear skies and h_can < 60")
    #group_tmp = group[ 
    #                  (group.seg_snow == 'snow free land') 
    #                 ].sample(frac=SAMP_FRAC)

    #prop_of_df = round(len(group_tmp) / len(atl08), 3)

    #print(f"Proportion of {group_name} total satifying this filter: {prop_of_df}")
    #divider = make_axes_locatable(axes[i])
    #cax = divider.append_axes('right', size='5%', pad=0.05)
    scat = axes[i].scatter(x=group.lon, y=group.lat, c=group['h_can'], cmap = forest_ht_cmap, vmin=0, vmax=25, s=0.05)
    #subplt = group.plot(ax=axes[i], x='lon', y='lat', c='h_can', cmap = forest_ht_cmap, vmin=0, vmax=25, kind='scatter', s=0.05) # 
    axes[i].set_title(str(len(group))+" ATL08 obs. during "+group_name+"\nstrong beam, clear skies, snow-free land, h_can<"+str(H_CAN_THRESH)+", "+group_name
                      #+"\n"+str(round(prop_of_df*100,1)) +"% of data"
                      , loc='left')
    #cbar = plt.colorbar(scat, extend='max', spacing='proportional', orientation='vertical', shrink=0.7, format="%.0f")
    #cbar.set_label(label = 'Canopy height (m)', size=16)

    i+=1

plt.tight_layout()


In [ ]:
# Simple histogram plotting is **ridiculously** hard to get correct.
#
# here, i try to plot a timeline of the count of ATL08, by day and month, for each year.
f, ax = plt.subplots(1,1,figsize=(12,3))
#atl08['fid'].groupby([atl08["date"].dt.year, atl08["date"].dt.month, atl08["date"].dt.day]).count().plot(ax=ax, kind='bar')

df = atl08.sample(frac=1)
print(len(df))
for i, seg_name in enumerate(df.yr.unique()):
    #ax.hist(atl08[atl08.yr == seg_name].groupby([atl08["date"].dt.month, atl08["date"].dt.day]), bins=bins, alpha=0.5, label=seg_name)
    ax.hist(df[df.yr == seg_name].m, bins=12, alpha=0.5, label=seg_name)
    ax.set_title(f"Timeline (month) of number of obs.", loc='left')
    ax.legend()
    ax.set_xlim(1, 12)
f, ax = plt.subplots(1,1,figsize=(12,3)) 
for i, seg_name in enumerate(df.yr.unique()):
    #ax.hist(atl08[atl08.yr == seg_name].groupby([atl08["date"].dt.month, atl08["date"].dt.day]), bins=bins, alpha=0.5, label=seg_name)
    ax.hist(df[df.yr == seg_name].lon, bins=360, alpha=0.5, label=seg_name)
    ax.set_title(f"Spatial interval (longitude) of number of obs.", loc='left')
    ax.legend()
    ax.set_xlim(-180, 180)

if False:
    # Good luck with getting this plot...
    f, ax = plt.subplots(2,1,figsize=(12,3), sharex=True, sharey=True)
    i=0
    for group_name, group in atl08.groupby('yr'):
        group['m'].groupby([group["date"].dt.month, group["date"].dt.day]).count().plot(ax=ax[i], kind='bar')
        ax[i].set_title(group_name, 
                        loc='left')
        i+=1
#plt.xlabel('Date', fontsize=12)
#plt.xticks(fontsize=7, rotation=90)

In [ ]:
# Use the same bins for each
xmin = atl08.n_toc_ph.min()
xmax = atl08.n_toc_ph.max()
bins = np.linspace(xmin, xmax, 300)

# Set up correct number of subplots, space them out. 
fig, ax = plt.subplots(figsize=(12,3))

for i, seg_name in enumerate(atl08.night_flg.unique()):
    ax.hist(atl08[atl08.night_flg == seg_name].n_toc_ph, bins=bins, alpha=0.5, label=seg_name)
    ax.set_title("Distribution of number of top-of-canopy photons per obs.", loc='left')
    ax.legend()

# same xlim for each so we can see differences
ax.set_xlim(0, 150)


In [ ]:
# CREATE GEOPANDAS DATAFRAME WITH SPATIAL REF and sample a fractions of it.
#
SAMP_FRAC=1
atl08_gdf = GeoDataFrame(atl08, geometry=gpd.points_from_xy(atl08.lon, atl08.lat), crs='epsg:4326').sample(frac=SAMP_FRAC)
print(f"There are {atl08_gdf.shape[0]} ATL08 observations in this sampled ({100*SAMP_FRAC}%) Geodataframe.")
print(len(atl08_gdf.columns))
xmin, ymin, xmax, ymax = atl08_gdf.total_bounds

# Subset to save for testing
bbox_sub = [-102.7684,40,-97,32037,47.48166]
atl08_gdf_sub = atl08_gdf.cx[bbox_sub[0]:bbox_sub[2], bbox_sub[1]:bbox_sub[3]]
atl08_gdf_sub.to_csv(os.path.split(CSV_TO_DIR)[0]+"/atl08_gdf_sub.csv", index=False, encoding="utf-8-sig")

## Plot Filtered ICESat-2 Data for Domain

In [ ]:
# Define a projection for the maps and the geodataframe

# Compare two equal area prjs
# https://map-projections.net/compare.php?p1=albers-equal-area-conic&p2=azimutal-equal-area-gpolar

# These albers prjs dont split the continents polygon well.
# boreal albers projection
#boreal_alb = "+proj=aea +lat_1=50 +lat_2=70 +lat_0=60 +lon_0=-170 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs"
#na_alb =     "+proj=aea +lat_1=50 +lat_2=70 +lat_0=60 +lon_0=-110 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs"



# https://proj.org/operations/projections/stere.html
boreal_stero = "+proj=stere +lat_0=90 +lat_ts=71 +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0"

# https://proj.org/operations/projections/laea.html
northpole_laea = "+proj=laea +lat_0=60 +lon_0=-180 +x_0=90 +y_0=0 +ellps=GRS80" # +datum=NAD83 +units=m +no_defs"

proj = northpole_laea

#if atl08_gdf.lon.min() < 0:
#    proj_alb = boreal_alb

# Clip world to ATL08 gdf
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world_atl08 = world.cx[xmin:xmax, ymin:ymax]
atl08_gdf_chull = atl08_gdf.unary_union.convex_hull

world_atl08 = world[world.intersects(atl08_gdf_chull)]
#NA = world[world['continent'] == 'North America'].to_crs(boreal_alb)

atl08_gdf_aea = atl08_gdf.to_crs(proj)
world_atl08_aea = world_atl08.to_crs(proj)

In [ ]:
# Exploring ATL08 with hexbins
# All years
ax_map_title = "ICESat-2 bin median canopy height "
cbar_map_title = 'Canopy height [m] (median, ATL08:h_can)'

d = datetime.date.today().strftime("%Y%b%d")
# Set up correct number of subplots, space them out. 
fig, ax = plt.subplots(1,1, figsize=(14,10), sharex=True, sharey=True)

bbox = atl08_gdf_aea.total_bounds

group_tmp = atl08_gdf_aea[atl08_gdf_aea.seg_snow == 'snow free land'] 

world_atl08_aea.plot(ax=ax, facecolor='grey', edgecolor='black',  alpha=0.5)
hb = ax.hexbin(group_tmp.geometry.x, group_tmp.geometry.y, C=group_tmp['h_can'], 
                   reduce_C_function=np.median, gridsize=250, cmap=forest_ht_cmap, vmax=25, mincnt=1, alpha=0.7)
world_atl08_aea.plot(ax=ax, facecolor='None', edgecolor='black',  alpha=0.9)

cbar = plt.colorbar(hb, extend='max', spacing='proportional', orientation='vertical', shrink=0.7, format="%.0f")
cbar.set_label(label = cbar_map_title, size=16)
   
ax.set_xlim(bbox[[0,2]])
ax.set_ylim(bbox[[1,3]])
ax.set_title(ax_map_title+"2019-2020", size=20, loc='left')
ax.grid()
fig_fn = os.path.join(DPS_OUTPUT_DIR, 'atl08_alb_h_can_2019_2020_'+d+'.png')
print(fig_fn)
#plt.savefig(fig_fn)

#ax.set(aspect='equal')

In [ ]:
print(ctx.providers.keys())
print(ctx.providers.Stamen.keys())

In [ ]:
# Exploring ATL08 with hexbins

def add_basemap(ax, crs, zoom='auto'):
    ctx.add_basemap(ax=ax, crs=crs, source=ctx.providers.Stamen.TerrainBackground, zoom=zoom) # ctx.sources.ST_TERRAIN
    #Create a scalebar object, with scaling factor of 1.0 px, since we're using projected coordinate system with unit 1 m
    #scalebar = ScaleBar(1.0)
    #Add scalebar to axes
    #ax.add_artist(scalebar)

bbox = atl08_gdf_aea.total_bounds

# Set up correct number of subplots, space them out. 
fig, ax = plt.subplots(ncols=atl08_gdf_aea.yr.nunique(), figsize=(30,10), sharex=True, sharey=True)

print(f"Filtering by year for clear skies")
for i, (lab, group) in enumerate(atl08_gdf_aea.groupby('yr')):
    
    world_atl08_aea.plot(ax=ax[i], facecolor='grey', edgecolor='black',  alpha=0.5)
    
    group_tmp = group[group.msw_flg == 0]
    
    hb = ax[i].hexbin(group_tmp.geometry.x, group_tmp.geometry.y, C=group_tmp['h_can'], reduce_C_function=np.median, gridsize=60, cmap=forest_ht_cmap, vmax=30, mincnt=1, alpha=0.7)
    
    #world_atl08_aea.plot(ax=ax[i], facecolor='None', edgecolor='black',  alpha=0.9)
    
    ax[i].set_xlim(bbox[[0,2]])
    ax[i].set_ylim(bbox[[1,3]])
    #ax[i].set_title(ax_map_title+lab, size=20, loc='left')
    #ax[i].grid()
    ax[i].set(aspect='equal')
    add_basemap(ax[i], crs=proj)
    
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.01, 0.5])
#cbar = fig.colorbar(hb, cax=cbar_ax)
#fig.colorbar(im, ax=axes.ravel().tolist())
cbar = plt.colorbar(hb, ax=ax.ravel().tolist(), extend='max', spacing='proportional', orientation='vertical', shrink=1, format="%.0f")
cbar.set_label(label = cbar_map_title, size=14)
#plt.tight_layout() 

In [ ]:
# Get a DEM for study domain
import os
import requests
import rasterio as rio
from rasterio import plot

#bbox_ll = [-150, 55, -115, 70]
#demtype = "SRTMGL3"
#url="https://portal.opentopography.org/API/globaldem?demtype={}&west={}&south={}&east={}&north={}&outputFormat=GTiff".format(demtype,*bbox_ll)
#response = requests.get(url)
#src = rio.open(response.content)